# Assemble MX Image Stack
* Assembling the time series of images, including both channels (red=nuclei and green=dead cell marker) for one plate/well
* Produces a set of images to generate an mp4 using ffmpeg
* Using only z-position=0

In [70]:
# Plate format: V3-21T-D4_E06 and V3-21T-D5_J12 (plate-name_well)
TOPDIR = '/mnt/monica/quaranta2/VDIPRR/VLW001/VLW_V23b/images'
FINFO = '/mnt/monica/quaranta2/VDIPRR/VLW001/VLW_V23b/VLW_V23bimageFileInfo.csv'
PNAME = 'V3-21T-D5'
WELL = 'J12'
SAVEDIR = '/home/monica/img_stack/im_stack2'
OWRITE = 'False'
RED_CH = 'Texas Red'
GR_CH = 'FITC'

In [71]:
import pandas as pd
import numpy as np
import mahotas as mh
from tifffile import imsave
import os
import re
import sys


# For the VLW23b fileinfo format
def getStackFilenames(root_dir, finfo, plate_name, well, red_channel='Cy3', green_channel='FITC'):
    pname = pd.Series([plate_name in x for x in finfo['plate_name']])
    redCh = finfo['channel'] == red_channel
    grCh = finfo['channel'] == green_channel
    well = finfo['well'] == well
    
    redInfo = finfo[pname & redCh & well].copy()
    redInfo = redInfo[redInfo['z_pos'] == 0]
    grInfo = finfo[pname & grCh & well].copy()
    grInfo = grInfo[grInfo['z_pos'] == 0]
    redPlateDir = ['Plate'+ str(p) for p in redInfo['plate_id']]
    grPlateDir = ['Plate'+ str(p) for p in grInfo['plate_id']]
    redFname = redInfo['file_name']
    grFname = grInfo['file_name']
    
    if(len(redPlateDir)==len(redFname)) and (len(grPlateDir)==len(grFname)):
        redPath = [os.path.join(root_dir, redPlateDir[i], redFname.array[i]) for i in range(len(redPlateDir))]
        grPath = [os.path.join(root_dir, grPlateDir[i], grFname.array[i]) for i in range(len(grPlateDir))]
        
        out = pd.DataFrame({plate_name + '_red': redPath,
                            plate_name + '_green': grPath})
        return(out)
    else:
        raise RuntimeError('Error occurred during stack filename creation.')

# Modified for the VLW23a fileinfo format; already has full image paths
def modifiedStackFilenames(finfo, plate_name, well, red_channel='nuc', green_channel='ch2'):
    pname = pd.Series([plate_name in x for x in finfo['plate_name']])
    redCh = finfo['channel'] == red_channel
    grCh = finfo['channel'] == green_channel
    well = finfo['well'] == well
    
    redInfo = finfo[plate_name & redCh & well].copy()
    grInfo = finfo[pname & grCh & well].copy()
    redPath = redInfo['file_name']
    grPath = grInfo['file_name']
    
    out = pd.DataFrame({plate_name + '_red': redPath,
                        plate_name + '_green': grPath})
    return(out)

    
if os.path.isfile(FINFO):
    file_info = pd.read_csv(FINFO, low_memory=False)
else:
    raise RuntimeError(f'Could not find file info file in {FINFO}')
    
# Column names of files saved by R have periods in them; replace with underscores
file_info.columns = [s.replace('.','_') for s in file_info.columns]

# The standard file info format
file_paths = getStackFilenames(TOPDIR, file_info, PNAME, WELL, RED_CH, GR_CH)

# The VLWa file info format
# file_paths = modifiedStackFilenames(file_info, PNAME, WELL)

# Check whether files exist; If not, may have been converted to png format
# Check red channel images
if not np.all([os.path.isfile(x) for x in file_paths[PNAME + '_red'].to_list()]):
    tempfn = os.path.basename(file_paths[PNAME + '_red'].to_list()[0])
    print(f'Could not find files (e.g. {tempfn}); Looking for png files instead')
    # try to replace file type with png
    file_paths[PNAME + '_red'] = [re.sub('tif$','png', fp) for fp in file_paths[PNAME + '_red'].to_list()]
    if not np.all([os.path.isfile(x) for x in file_paths[PNAME].to_list()]):
        tempfn = os.path.basename(file_paths[PNAME + '_red'].to_list()[0])
        raise RuntimeError(f'Could not find files, e.g. {tempfn}')

# Check green channel images
if not np.all([os.path.isfile(x) for x in file_paths[PNAME + '_green'].to_list()]):
    tempfn = os.path.basename(file_paths[PNAME + '_green'].to_list()[0])
    print(f'Could not find files (e.g. {tempfn}); Looking for png files instead')
    # try to replace file type with png
    file_paths[PNAME + '_green'] = [re.sub('tif$','png', fp) for fp in file_paths[PNAME + '_green'].to_list()]
    if not np.all([os.path.isfile(x) for x in file_paths[PNAME].to_list()]):
        tempfn = os.path.basename(file_paths[PNAME + '_green'].to_list()[0])
        raise RuntimeError(f'Could not find files, e.g. {tempfn}')

im_stack = np.array([mh.imread(f) for f in file_paths[PNAME + '_red']],np.uint16)
gr_stack = np.array([mh.imread(f) for f in file_paths[PNAME + '_green']], np.uint16)

# Output files
out_f1 = os.path.join(SAVEDIR, 'red_stack.tif')
out_f2 = os.path.join(SAVEDIR, 'green_stack.tif')

if OWRITE == 'False':
    i = 0
    j = 0
    while os.path.isfile(out_f1):
        i += 1
        out_base = os.path.dirname(out_f1)
        fn = os.path.basename(out_f1).split('.')[0] + '_' + str(i) + '.' + os.path.basename(out_f1).split('.')[1]
        out_f1 = os.path.join(out_base, fn)
    while os.path.isfile(out_f2):
        j += 1
        out_base = os.path.dirname(out_f2)
        fn2 = os.path.basename(out_f2).split('.')[0] + '_' + str(j) + '.' + os.path.basename(out_f2).split('.')[1]
        out_f2 = os.path.join(out_base, fn2)   

imsave(out_f1, im_stack)
imsave(out_f2, gr_stack)

print(f'File saved: {out_f1}')
print(f'File saved: {out_f2}')

File saved: /home/monica/img_stack/im_stack2/red_stack.tif
File saved: /home/monica/img_stack/im_stack2/green_stack.tif


### Rescaling the channels

In [72]:
from skimage.exposure import rescale_intensity
from numpy import percentile


def rescaleImg(img, pct=[2,98], lh=[0,0]):
    if lh == [0,0]:
        plow, phi = percentile(img, (pct[0], pct[1]))
    elif len(lh) == 2 and lh != [0,0]:
        plow = lh[0]
        phi = lh[1]
    else:
        print('ERROR: rescaleImg arg <lh> requires a tuple of len 2')
        return(img)
    img_rescale = rescale_intensity(img, in_range=(plow, phi))
    return img_rescale

In [73]:
scaled_red = np.copy(im_stack)
for i in range(len(im_stack)):
    scaled_red[i] = rescaleImg(im_stack[i])

print(im_stack.shape)
print(scaled_red.shape)

(6, 2160, 2160)
(6, 2160, 2160)


In [74]:
imsave(f'{SAVEDIR}/scaled_red.tif', scaled_red)

In [75]:
scaled_gr = np.copy(gr_stack)
for i in range(len(gr_stack)):
    scaled_gr[i] = rescale_intensity(gr_stack[i],in_range=(1400, 5000))
    
print(gr_stack.shape)
print(scaled_gr.shape)

(6, 2160, 2160)
(6, 2160, 2160)


In [76]:
imsave(f'{SAVEDIR}/scaled_gr.tif', scaled_gr)

### Generate H.264 encoded mp4 from image stacks
* Concatenate images so red channel is on the left and green channel is on the right
* Save the concatenated images in a new directory so they can be easily read in by ffmpeg

Save just concatenated images; No border or text added

In [68]:
# Create ffmpeg_imgs directory in savedir
if not os.path.exists(f'{SAVEDIR}/ffmpeg_imgs'):
    os.makedirs(f'{SAVEDIR}/ffmpeg_imgs')

for i in range(len(scaled_red)):
    red = scaled_red[i].copy()
    gr = scaled_gr[i].copy()
    new = np.concatenate((red, gr), axis=1)
    imsave(f'{SAVEDIR}/ffmpeg_imgs/image{i}.tif', new)

Save images with a black border and text labelling the red and green channels

In [77]:
import cv2 


font = cv2.FONT_HERSHEY_DUPLEX

for i in range(len(scaled_red)):
    red = scaled_red[i].copy()
    gr = scaled_gr[i].copy()
    new = np.concatenate((red, gr), axis=1)
    border = np.pad(new, pad_width=50, mode='constant', constant_values=0)
    imsave(f'{SAVEDIR}/ffmpeg_imgs/image{i}.tif', border)
    im = cv2.imread(f'{SAVEDIR}/ffmpeg_imgs/image{i}.tif')
    cv2.putText(im, 'nuclei', (50,2240), font, 1, (0, 0, 255), 2)
    cv2.putText(im, 'dead', (2225,2240), font, 1, (0, 255, 0), 2)
    cv2.imwrite(f'{SAVEDIR}/ffmpeg_imgs/image{i}.tif', im)

Then run this command in ffmpeg_imgs directory:  
``` ffmpeg -r 2 -f image2 -s 2160x4320 -i image%d.tif -vcodec libx264 -crf 25  -pix_fmt yuv420p output.mp4 ```  